In [37]:
import pandas as pd
df_idf= pd.read_csv('./data/tax.csv')

In [38]:
df_idf= df_idf[df_idf["Text"].str.contains("Duitse vertaling")==False]

In [6]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf['text'] = df_idf['Text']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the first 'text'
df_idf['text'][4]

'nl fr belgiëlex be kruispuntbank wetgeving raad van state eli navigatie systeem via een europese identificatiecode voor wetgeving http www ejustice just fgov be eli besluit staatsblad einde eerste woord laatste woord publicatie numac federale overheidsdienst financien januari koninklijk besluit tot remediering van het dubbelgebruik van artikel quater in het kb wib filip koning der belgen aan allen die nu zijn en hierna wezen zullen onze groet gelet op artikel van het wetboek van de inkomstenbelastingen ingevoegd bij de wet van december gelet op het kb wib gelet op het koninklijk besluit van november betreffende de administratieve en begrotingscontrole artikelen en overwegende dat de bepalingen van dit besluit geen rechtstreekse of onrechtstreekse financiële weerslag kunnen hebben gelet op de wetten op de raad van state gecoördineerd op januari artikel overwegende dat de bepalingen van dit besluit niet reglementair van aard zijn overwegende dat artikel van het koninklijk besluit van de

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')
from nltk.corpus import stopwords
#load a set of stop words
stop_words= stopwords.words("dutch")
stop_words.extend(['bis', 'NL', 'FR','artikel', "januari", "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "oktober", "november", "december"])

#get the text column 
docs=df_idf['text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stop_words)
word_count_vector=cv.fit_transform(docs)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahboubeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahboubeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Mahboubeh\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fr', 'nl'] not in stop_words.
  warnings.warn(


In [8]:
word_count_vector.shape

(662, 20399)

In [9]:
list(cv.vocabulary_.keys())[:10]

['belgiëlex',
 'be',
 'kruispuntbank',
 'wetgeving',
 'raad',
 'state',
 'eli',
 'navigatie',
 'systeem',
 'via']

In [10]:
list(cv.get_feature_names())[2000:2015]

C:\Users\Mahboubeh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['beleggingsvennootschap',
 'beleggingsvennootschappen',
 'beleggingswaarde',
 'beleid',
 'beleids',
 'beleidsadviseurs',
 'beleidsdomein',
 'beleidsdomeinen',
 'beleidsexpertise',
 'beleidsfocus',
 'beleidsgeorienteerde',
 'beleidsgroep',
 'beleidsinstrumenten',
 'beleidskeuze',
 'beleidslijnen']

# TfidfTransformer to Compute Inverse Document Frequency (IDF)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [12]:
tfidf_transformer.idf_

array([2.59150021, 6.80362781, 6.80362781, ..., 5.70501552, 4.85771766,
       6.3981627 ])

# Computing TF-IDF and Extracting Keywords

In [20]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_csv('./data/tax.csv')
df_test= df_test[df_test["Text"].str.contains("Duitse vertaling")==False]

df_test['text'] = df_test['Text']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_title=df_test['Title'].tolist()

In [21]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [22]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[4]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Title=====")
print(docs_title[4])
print("\n=====Body=====")
print(docs_test[4])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====
SERVICE PUBLIC FEDERAL FINANCES
Administration générale de la Fiscalité. - Impôts sur les revenus. - Avis déterminant un nouveau modèle d'attestation n°  281.85 permettant d'apporter les preuves requises en matière de réduction d'impôt pour l'acquisition de nouvelles actions ou parts d'entreprises qui débutent conformément à l'article 14526,  §§  4 à 6, du Code des impôts sur les revenus 1992 (CIR 92) et à l'article 6312/1, de l'arrêté royal d'exécution du CIR 92 (AR/CIR 92). - Cet avis modifie celui paru au Moniteur belge du 16.02.2018, p. 4487.

=====Body=====
nl fr einde eerste woord laatste woord publicatie numac federale overheidsdienst financien algemene administratie van de fiscaliteit inkomstenbelastingen bericht tot vaststelling van een nieuw model van attest nr dat toelaat het vereiste bewijs te leveren inzake de belastingvermindering voor de verwerving van nieuwe aandelen van startende ondernemingen overeenkomstig artikel t e m van het wetboek van de inkomst

In [23]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_test[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [24]:
idx=120
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
SERVICE PUBLIC FEDERAL INTERIEUR
5 JUIN 2020. - Arrêté ministériel modifiant l'arrêté ministériel du 23 mars portant des mesures d'urgence pour limiter la propagation du coronavirus COVID-19. - Errata, p. 41980.

=====Body=====
nl fr einde eerste woord laatste woord publicatie numac federale overheidsdienst binnenlandse zaken juni ministerieel besluit houdende wijziging van het ministerieel besluit van maart houdende dringende maatregelen om de verspreiding van het coronavirus covid te beperken errata in het belgisch staatsblad nr van juni vierde editie bladzijde moeten de woorden wordt een artikel ter toegevoegd in artikel al vervangen worden door wordt een artikel quater toegevoegd op bladzijde wordt in artikel het woord zijn na de woorden zijn de maatregelen voorzien in dit besluit geschrapt in de nederlandstalige tekst begin eerste woord laatste woord publicatie numac 

===Keywords===
bladzijde 0.45
toegevoegd 0.336
errata 0.3
nederlandstalige 0.252
woorden 0.25
mi

# Generate keywords for a batch of documents

In [25]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    
    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
df

,doc,keywords
0,nl fr belgiëlex be kruispuntbank wetgeving raa...,"{'familiale': 0.427, 'successierechten': 0.352..."
1,nl fr einde eerste woord laatste woord publica...,"{'onderwijs': 0.375, 'wetenschappen': 0.346, '..."
2,nl fr belgiëlex be kruispuntbank wetgeving raa...,"{'wib': 0.412, 'kb': 0.407, 'groepsbijdrage': ..."
3,nl fr belgiëlex be kruispuntbank wetgeving raa...,"{'quater': 0.452, 'aanduiding': 0.368, 'gedele..."
4,nl fr einde eerste woord laatste woord publica...,"{'vennootschap': 0.344, 'startende': 0.317, 'a..."
...,...,...
657,nl fr belgiëlex be kruispuntbank wetgeving raa...,"{'eu': 0.349, 'crowdfundingdienstverleners': 0..."
658,nl fr belgiëlex be kruispuntbank wetgeving raa...,"{'werkloosheid': 0.366, 'koninklijk': 0.308, '..."
659,nl fr belgiëlex be kruispuntbank wetgeving raa...,"{'verzekering': 0.385, 'geneeskundige': 0.321,..."
660,nl,{}


In [33]:
df["keywords"][661]

{'hoppinpunten': 0.578,
 'hoppinpunt': 0.3,
 'vlaamse': 0.299,
 'mobiliteitsbeleid': 0.187,
 'besluit': 0.187,
 'netwerklogica': 0.185,
 'vervoerregioraad': 0.162,
 'vermeld': 0.16,
 'interregionale': 0.116,
 'regering': 0.105}

In [39]:
df["doc"][661]

'nl fr belgiëlex be kruispuntbank wetgeving raad van state eli navigatie systeem via een europese identificatiecode voor wetgeving http www ejustice just fgov be eli besluit staatsblad einde eerste woord laatste woord publicatie numac vlaamse overheid februari besluit van de vlaamse regering over de hoppinpunten en tot wijziging van het besluit van de vlaamse regering van januari tot bepaling van de nadere regels betreffende de financiering en de samenwerking voor het mobiliteitsbeleid rechtsgronden dit besluit is gebaseerd op het decreet van april betreffende de basisbereikbaarheid artikel en tweede lid vormvereisten de volgende vormvereisten zijn vervuld de vlaamse minister bevoegd voor de begroting heeft zijn akkoord gegeven op juli de mobiliteitsraad heeft advies gegeven op september de raad van state heeft advies gegeven op december met toepassing van artikel eerste lid van de wetten op de raad van state gecoördineerd op januari initiatiefnemer dit besluit wordt voorgesteld door d